In [5]:
import json
import serial
import time
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt

BASIC_SPO2_LEVEL = 97
BASIC_TEMP_VALUE = 36.6
DATA_ACQUISITION_TIME_S = 30  # 10
SERIAL_INTERFACE = "COM4"

data = []
if DATA_ACQUISITION_TIME_S <= 0:
    # Reading data from file
    with open('data.json') as file:
        data = json.load(file)

else:
    # Reading data from Arduino serial
    try:
        com = serial.Serial(SERIAL_INTERFACE, 115200, timeout=1)
    except serial.SerialException:
        print("Serial connection on", SERIAL_INTERFACE, "cannot be opened")
        exit()

    time.sleep(1)
    print("Serial connection open")
    end_timestamp = datetime.now().timestamp() + DATA_ACQUISITION_TIME_S

    print("Reading data for", DATA_ACQUISITION_TIME_S, "seconds...")
    while end_timestamp - datetime.now().timestamp() > 0:
        try:
            line = com.readline()
            data.append(np.fromstring(line.decode("utf-8"), dtype=float, sep=" "))
        except UnicodeDecodeError:
            continue

    print("Serial connection closed")
    com.close()

Serial connection open
Reading data for 30 seconds...
Serial connection closed


In [6]:
data = np.array(data)
data

array([[ 0.000e+00,  4.300e+01,  2.807e+01, -1.000e+00, -1.000e+00],
       [ 1.000e+00,  2.313e+01,  2.000e-02, -1.600e-01,  1.005e+01],
       [ 1.000e+00,  2.308e+01,  2.000e-02, -1.500e-01,  1.007e+01],
       ...,
       [ 1.000e+00,  2.305e+01,  0.000e+00, -1.600e-01,  1.000e+01],
       [ 1.000e+00,  2.304e+01,  1.000e-02, -1.600e-01,  1.001e+01],
       [ 1.000e+00,  2.296e+01,  2.000e-02, -1.600e-01,  1.001e+01]])

Dane przychodzą teraz w formacie:

```
0 lm35_temp max_temp max_spo2 max_bpm
1 ada_temp ada_x ada_y ada_z
1 ada_temp ada_x ada_y ada_z
1 ada_temp ada_x ada_y ada_z
1 ada_temp ada_x ada_y ada_z
...
```

Każda linia skąda się z 5 wartości numerycznych oddzielonych jedną spacją.  
Każda linia, gdzie pierwsza wartość to 0, przesyła wartości z termometru i pulsoksymetru  
Każda linia, gdzie pierwsza wartość to 1, przesyła wartości z akcelerometru  
Na jedną linię `0` przypada 200 linni `1`, teoretycznie przesył danych zawsze zaczyna się od linni `0` bo arduino się restartuje przy otwarciu COM4, ale nie gwarantuję tego  
Dokładny format podany powyżej  

Poniżej operacje numpyowe do wydzielenia danych do osobnych tablic


In [7]:
max = data[data[:, 0] == 0.0]
ada = data[data[:, 0] == 1.0]

lm35_temp = max[:, 1]
max_temp = max[:, 2]
max_spo2 = max[:, 3]
max_bpm = max[:, 4]

ada_temp = ada[:, 1]
ada_x = ada[:, 2]
ada_y = ada[:, 3]
ada_z = ada[:, 4]

In [11]:
print(lm35_temp)
print(max_temp)
print(max_spo2)
print(max_bpm)

[ 43.  47.  43. 135.  48.  48.  48.]
[28.07 28.11 28.13 28.13 29.03 29.04 28.08]
[-1. -1. -1. -1. -1. -1. -1.]
[-1. -1. -1. -1. -1. -1. -1.]


In [19]:
print(len(ada_z))
ada_z

1226


array([10.05, 10.07, 10.06, ..., 10.  , 10.01, 10.01])

In [14]:
import pandas as pd

pd.to_pickle(data, "data.pkl")

In [18]:
type(pd.read_pickle("data.pkl"))

numpy.ndarray